In the previous notebook we created TFrecords from the bible. In the notebook we'll take a look at them and then build a model that predicts the Book of the bible a verse came from

In [1]:
import tensorflow as tf
from preppy import Preppy, RankPreppy

Make a dataset by reading the train 

In [2]:
tf.reset_default_graph()
dataset = tf.data.TFRecordDataset(['./train.tfrecord']).map(RankPreppy.parse)

In [3]:
iterator = dataset.make_one_shot_iterator()
next_item = iterator.get_next()

In [4]:
sess =tf.InteractiveSession()

In [ ]:
sess.run(next_item)

In [ ]:
dataset.output_shapes

In [ ]:
def expand(x):
    x['length_q'] = tf.expand_dims(tf.convert_to_tensor(x['length_q']),0)
    x['length_a'] = tf.expand_dims(tf.convert_to_tensor(x['length_a']),0)
    x['label'] = tf.expand_dims(tf.convert_to_tensor(x['label']),0)
    x['user'] = tf.expand_dims(tf.convert_to_tensor(x['user']),0)
    return x
def deflate(x):
    x['length_q'] = tf.squeeze(x['length_q'])
    x['length_a'] = tf.squeeze(x['length_a'])
    x['label'] = tf.squeeze(x['label'])
    x['user'] = tf.squeeze(x['user'])
    return x



In [ ]:
batch_iter = dataset.map(expand).padded_batch(32,padded_shapes={
    "label":1,
    "length_q":1,
    "length_a":1,
    "user":1,
    "query":tf.TensorShape([None]),
    "response":tf.TensorShape([None])
}).map(deflate)
next_item = batch_iter.repeat().make_one_shot_iterator().get_next()

In [ ]:
sess.run(next_item)

In [ ]:
class Model():
    def __init__(self,inputs):
        query =  inputs['query']
        response =  inputs['response']
        length_q = inputs['length_q']
        length_a = inputs['length_a']
        label = inputs['label']
        user = inputs['user']
        
        self.lr = tf.placeholder(shape=None,dtype=tf.float32)
        
        emb_vec_word = tf.get_variable("emb_word",dtype=tf.float32,shape=[74,32])
        emb_vec_user = tf.get_variable("emb_user",dtype=tf.float32,shape=[101,50])
        emb_query = tf.nn.embedding_lookup(emb_vec_word, query)
        emb_response = tf.nn.embedding_lookup(emb_vec_word, response)
        emb_user = tf.nn.embedding_lookup(emb_vec_user, user)
        
        cell = tf.nn.rnn_cell.GRUCell(128)
        outputs, state = tf.nn.dynamic_rnn(cell,emb_source,dtype=tf.float32,sequence_length=lengths)
        
        book_logits =  tf.contrib.layers.fully_connected(state,num_outputs=64,activation_fn=tf.tanh)
        book_logits =  tf.contrib.layers.fully_connected(state,num_outputs=215,activation_fn=None)
        
        loss = tf.losses.sparse_softmax_cross_entropy(book_id,book_logits)
        self.loss = tf.reduce_mean(loss)
        opt = tf.train.AdamOptimizer(self.lr)
        self.train = opt.minimize(self.loss)


In [ ]:
M = Model(next_item)
sess.run(tf.global_variables_initializer())

In [ ]:
num =1
import sys
while True:
    try:
        _,loss = sess.run([M.train,M.loss],feed_dict={M.lr:0.0001})
        num+=1
        sys.stdout.write("\r" + str(loss))
        sys.stdout.flush()
    except:
        pass

    